## setup/installation

In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 28.4 MB/s eta 0:00:00


In [ ]:
!pip install streamlit openai llama-index nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install llama-index-experimental

## import dependencies

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
)
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager

## ingest data

### wikipedia articles

In [ ]:
wiki_titles = [
    "Crushing_plant",
    "Crusher",
    # "AUBEMA_Maschinenfabrik_GmbH",
    "High-frequency_vibrating_screens",
    # "Thyssenkrupp_Polysius_GmbH",
    "Thyssenkrupp",
]

data_wiki_path = "data/data_wikis"

In [ ]:
from pathlib import Path

import requests



for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    print(page)
    wiki_text = page["extract"]

    data_path = Path(data_wiki_path)
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

{'pageid': 38269892, 'ns': 0, 'title': 'Crushing plant', 'extract': 'A Crushing plant is one-stop crushing installation, which can be used for rock crushing, garbage crushing, building materials crushing and other similar operations. Crushing plants may be either fixed or mobile. A crushing plant has different stations (primary, secondary, tertiary, ...) where different crushing, selection and transport cycles are done in order to obtain different stone sizes or the required granulometry.\n\n\n== Components ==\nCrushing plants make use of a large range of equipment, such as a pre-screener, loading conveyor, intake hopper, magnetic separator, crushing unit, such as jaw crushers and cone crusher etc.\n\nVibration feeder: These machines feed the jaw and impact crusher with the rocks and stones to be crushed.\nCrushers: These are the machines where the rocks and stones are crushed. There are different types of crushers for different types of rocks and stones and different sizes of the inpu

In [ ]:
# Load all wiki documents
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"{data_wiki_path}/{wiki_title}.txt"]
    ).load_data()

### load csv excel using pandas

In [ ]:
summary_csv_path = "data/Crusher_data.csv"

In [ ]:
df = pd.read_csv(summary_csv_path)

In [ ]:
df

,Hersteller,Typenbezeichnung,Durchsatz min [t/h],Durchsatz max [t/h],Durchsatz min [m³/h],Durchsatz max [m³/h],Leistung min [kW],Leistung max [kW]
0,NaN,NaN,min.,max.,NaN,max.,min.,max.
1,NaN,NaN,[t/h],[mm],NaN,[t/h],[kW],NaN
2,Beyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Brechanlage,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,KB 500,NaN,NaN,NaN,20,NaN,22
...,...,...,...,...,...,...,...,...
1367,NaN,WB 2000/1800 - 2800,NaN,NaN,NaN,NaN,440,NaN
1368,NaN,WB 800/600 - 1600,NaN,NaN,NaN,NaN,44,NaN
1369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### load all files in directory

In [ ]:
# only load markdown files
required_exts = [".csv", ".pdf", "xlsx"]

reader = SimpleDirectoryReader(
    input_dir="data",
    # required_exts=required_exts,
    recursive=True,
)

In [ ]:
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

Loaded 4 docs


## define the global models and embeddings

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXf"

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [ ]:
import logging
import sys
from IPython.display import Markdown, display

import pandas as pd
from llama_index.experimental.query_engine import PandasQueryEngine
# from llama_index.pandas_query_engine import PandasQueryEngine

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
# defining again here
summary_csv_path = "data/Crusher_data.csv"
df = pd.read_csv(summary_csv_path)

In [ ]:
query_engine = PandasQueryEngine(df=df, verbose=True)

In [ ]:
response = query_engine.query(
    "which types have throughput of 50 t/h",
)

> Pandas Instructions:
```
df[(df['Durchsatz min [t/h]'] == 50) | (df['Durchsatz max [t/h]'] == 50)]
```
> Pandas Output: Empty DataFrame
Columns: [Hersteller, Typenbezeichnung, Durchsatz min [t/h], Durchsatz max [t/h], Durchsatz min [m³/h], Durchsatz max [m³/h], Leistung min [kW], Leistung max [kW]]
Index: []


#### load documents from local directory

In [ ]:
documents = SimpleDirectoryReader("data").load_data()

In [ ]:
print(f"Loaded {len(documents)} docs")

Loaded 66 docs


In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query = "what does the data describe in general. give a brief overview"

query_engine = index.as_query_engine()
response = query_engine.query(query)
print(response)

The data appears to describe various types of crushers and their performance metrics, such as throughput, power consumption, and specific performance indicators related to crushing efficiency. It seems to provide a detailed analysis of different types of crushers and their operational characteristics for industrial purposes.


In [ ]:
query = "which type of machines all have throughput greater than 80 tons per hour. give the reference to the document from which this information was found"

query_engine = index.as_query_engine()
response = query_engine.query(query)
print(response)

The types of machines with throughput greater than 80 tons per hour are:
- TITAN® 56 D 75
- TITAN® 64 D 75
- TITAN® 72 D 100
- TITAN® 80 D 100
- TITAN® 88 D 100
- TITAN® 60 D 160
- TITAN® 70 D 160
- TITAN® 80 D 160
- TITAN® 90 D 160
- TITAN® 96 D 180
- TITAN® 108 D 180

This information was found in the document located at file_path: /content/data/Thyssen Hammer titan.pdf, specifically on page_label: 6.


In [ ]:
query = "what are all the recommended machines, conveyors and separators that should be used together which optimize high throughput and low power consumption. also give the references to the documents from which this information was found "

query_engine = index.as_query_engine()
response = query_engine.query(query)
print(response)

The recommended machines, conveyors, and separators that should be used together to optimize high throughput and low power consumption are the following:
1. Reversible single and double-shaft hammer mill with various drive concepts for optimal utilization of beater heads.
2. Mechanically adjustable grinding walls for wear compensation and achieving the desired final particle size.
3. Hydraulically adjustable grinding walls for varying tasks based on product quality and particle size distribution.
4. Pressure-resistant housing design for applications with explosive products.
5. Hammer mill with a rotating grinding track for very moist products prone to sticking.
6. Hammer mill with heated grinding walls (electric, steam, or thermal oil) to improve material flow for materials prone to sticking.
7. Hammer mill with exchangeable grate to limit oversize particles (not suitable for moist, sticky, and clay-like materials).
8. Grinding for drying to reduce material moisture while simultaneousl

## chat UI

In [ ]:
# import streamlit as st
# import openai
# from llama_index.llms.openai import OpenAI
# try:
#   from llama_index import VectorStoreIndex, ServiceContext, Document, SimpleDirectoryReader
# except ImportError:
#   from llama_index.core import VectorStoreIndex, ServiceContext, Document, SimpleDirectoryReader


In [ ]:
# st.set_page_config(page_title="Chat with docs from your company's technical resources, powered by RWTH, LlamaIndex and Streamlit", page_icon="🦙", layout="centered", initial_sidebar_state="auto", menu_items=None)
# openai.api_key = st.secrets.openai_key
# st.title("Chat with ")
# st.info("Check out the full tutorial to build this app in our [blog post](https://blog.streamlit.io/build-a-chatbot-with-custom-data-sources-powered-by-llamaindex/)", icon="📃")

# if "messages" not in st.session_state.keys(): # Initialize the chat messages history
#     st.session_state.messages = [
#         {"role": "assistant", "content": "Ask me a question about Streamlit's open-source Python library!"}
#     ]

# @st.cache_resource(show_spinner=False)
# def load_data():
#     with st.spinner(text="Loading and indexing the Streamlit docs – hang tight! This should take 1-2 minutes."):
#         reader = SimpleDirectoryReader(input_dir="./data", recursive=True)
#         docs = reader.load_data()
#         # llm = OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert o$
#         # index = VectorStoreIndex.from_documents(docs)
#         service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert on the Streamlit Python library and your job is to answer technical questions. Assume that all questions are related to the Streamlit Python library. Keep your answers technical and based on facts – do not hallucinate features."))
#         index = VectorStoreIndex.from_documents(docs, service_context=service_context)
#         return index

# index = load_data()

# if "chat_engine" not in st.session_state.keys(): # Initialize the chat engine
#         st.session_state.chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)

# if prompt := st.chat_input("Your question"): # Prompt for user input and save to chat history
#     st.session_state.messages.append({"role": "user", "content": prompt})

# for message in st.session_state.messages: # Display the prior chat messages
#     with st.chat_message(message["role"]):
#         st.write(message["content"])

# # If last message is not from assistant, generate a new response
# if st.session_state.messages[-1]["role"] != "assistant":
#     with st.chat_message("assistant"):
#         with st.spinner("Thinking..."):
#             response = st.session_state.chat_engine.chat(prompt)
#             st.write(response.response)
#             message = {"role": "assistant", "content": response.response}
#             st.session_state.messages.append(message) # Add response to message history

In [ ]:
!pip install -q streamlit
!pip install ipwhois
!npm install localtunnel

In [ ]:
!pip -q install pyngrok

#### the chat ui streamlit app

In [ ]:
%%writefile streamlit_chat_ui.py
import streamlit as st
import openai
from llama_index.llms.openai import OpenAI
try:
  from llama_index import VectorStoreIndex, ServiceContext, Document, SimpleDirectoryReader
except ImportError:
  from llama_index.core import VectorStoreIndex, ServiceContext, Document, SimpleDirectoryReader

system_prompt_ = "You are a technical research assistant in the domain of mining and mineral processing. \
                                            You speacialize in recommending technical design parameters and are an expert in giving suggestions based on the corpus of technical documents provided. \
                                            Be as helpful and descriptive as you can be. Always refer to the loaded technical documents and give references to the documents from where the information was found."

st.set_page_config(page_title="Chat with docs from your company's technical resources, powered by RWTH Aachen, LlamaIndex and Streamlit", page_icon="🦙", layout="centered", initial_sidebar_state="auto", menu_items=None)
openai.api_key = st.secrets.openai_key
st.title("Mining Crushing Plant Design Recommendation System Chatbot and Technical Assistant")
st.info("", icon="📃")

if "messages" not in st.session_state.keys(): # Initialize the chat messages history
    st.session_state.messages = [
        {"role": "assistant", "content": "Give a brief descriptive overview of the technical documents that you are working with now."}
    ]

@st.cache_resource(show_spinner=False)
def load_data():
    with st.spinner(text="Loading and indexing the technical docs – hang tight! This should take 1-2 minutes."):
        reader = SimpleDirectoryReader(input_dir="./data", recursive=True)
        docs = reader.load_data()
        # llm = OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert o$
        # index = VectorStoreIndex.from_documents(docs)
        service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-4", temperature=0.5, system_prompt=system_prompt_))
        index = VectorStoreIndex.from_documents(docs, service_context=service_context)
        return index

index = load_data()

if "chat_engine" not in st.session_state.keys(): # Initialize the chat engine
        st.session_state.chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)

if prompt := st.chat_input("Your question"): # Prompt for user input and save to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

for message in st.session_state.messages: # Display the prior chat messages
    with st.chat_message(message["role"]):
        st.write(message["content"])

# If last message is not from assistant, generate a new response
if st.session_state.messages[-1]["role"] != "assistant":
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            response = st.session_state.chat_engine.chat(prompt)
            st.write(response.response)
            message = {"role": "assistant", "content": response.response}
            st.session_state.messages.append(message) # Add response to message history

Overwriting streamlit_chat_ui.py


#### get colab notebook network info

In [ ]:
!curl https://loca.lt/mytunnelpassword

34.71.126.138

In [ ]:
# from ipwhois import IPWhois
# from requests import get

# ip = get('https://api.ipify.org').text
# whois = IPWhois(ip).lookup_rdap(depth=1)
# cidr = whois['network']['cidr']
# name = whois['network']['name']

# print('\n')
# print('Provider:  ', name)
# print('Public IP: ', ip)
# print('CIDRs:     ', cidr)



Provider:   GOOGL-2
Public IP:  34.71.126.138
CIDRs:      34.64.0.0/10


### run the app in the background

In [ ]:
!streamlit run /content/streamlit_chat_ui.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.191s
your url is: https://cool-jars-hammer.loca.lt
^C


#### ngrok based hosting | alternative to localtunnel

In [ ]:
#@title This last cell would keep the app running. If stoped, the app would be disconnected.
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print('Link to web app:')
print(public_url)
!streamlit run --server.port 80 streamlit_chat_ui.py >/dev/null

ERROR:pyngrok.process.ngrok:t=2024-05-25T13:28:30+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-05-25T13:28:30+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-05-25T13:28:30+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.